In [ ]:
# !pip install transformers accelerate==0.21.0 datasets evaluate rouge_score

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!unzip -q 'news_dataset.zip'

In [2]:
!nvidia-smi

Wed Oct 25 10:57:27 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                        Off| 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8               13W /  70W|      2MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
import pandas as pd
import os
from transformers import AutoTokenizer
import numpy as np
from datasets import Dataset
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [4]:
directory_path = 'BBC News Summary/News Articles/sport'

file_names = []
file_contents = []

for file_name in os.listdir(directory_path):
    if file_name.endswith('.txt'):
        file_names.append(file_name)
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, 'rb') as file:
            content = file.read().decode(errors='ignore')
            file_contents.append(content)

data = {'File_Name': file_names, 'News': file_contents}
df1 = pd.DataFrame(data)
file_names = []
file_contents = []

for file_name in os.listdir(directory_path):
    if file_name.endswith('.txt'):
        file_names.append(file_name)
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, 'rb') as file:
            content = file.read().decode(errors='ignore')
            file_contents.append(content)

data = {'File_Name': file_names, 'News': file_contents}
df1 = pd.DataFrame(data)

In [5]:
df1['News'][3]

'Hantuchova in Dubai last eight\n\nDaniela Hantuchova moved into the quarter-finals of the Dubai Open, after beating Elene Likhotseva of Russia 7-5 6-4, and now faces Serena Williams.\n\nAustralian Open champion Williams survived an early scare to beat Russia\'s Elena Bovina 1-6 6-1 6-4. World number one Lindsay Davenport and Anastasia Myskina also progressed. Davenport defeated China\'s Jie Zheng 6-2 7-5, while French Open champion Myskina sailed through after her opponent Marion Bartoli retired hurt. American Davenport will now face fellow former Wimbledon champion, Conchita Martinez of Spain, who ousted seventh-seeded Nathalie Dechy of France 6-1 6-2. Myskina will face eighth-seed Patty Schnyder from Switzerland, who defeated China\'s Li Na 6-3 7-6 (10-8). The other quarter final pits wild card Sania Mirza of India against Jelena Jankovic of Serbia and Montenegro, who both won on Tuesday.\n\nBefore her meeting with Martinez, Davenport believes there is some room for improvement in h

In [6]:
directory_path = 'BBC News Summary/Summaries/sport'
file_names = []
file_contents = []

for file_name in os.listdir(directory_path):
    if file_name.endswith('.txt'):
        file_names.append(file_name)
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, 'rb') as file:
            content = file.read().decode(errors='ignore')
            file_contents.append(content)
            
data = {'File_Name': file_names, 'Summary': file_contents}
df2 = pd.DataFrame(data)

In [7]:
df2['Summary'][3]

'Williams was also far from content.Davenport defeated China\'s Jie Zheng 6-2 7-5, while French Open champion Myskina sailed through after her opponent Marion Bartoli retired hurt."I don\'t know what I was doing there," she said."I feel I have an advantage (over Serena) because I have already played two matches on these courts," she said.Daniela Hantuchova moved into the quarter-finals of the Dubai Open, after beating Elene Likhotseva of Russia 7-5 6-4, and now faces Serena Williams."It was really windy and I hadn\'t played in the wind.But Hantuchova is in upbeat mood ahead of her clash with the younger Williams sister, who was handed a first-round bye.'

In [8]:
data = pd.merge(df1, df2, on='File_Name').drop(['File_Name'], axis= 1)
data.head()

,News,Summary
0,Robinson answers critics\n\nEngland captain Ja...,"Robinson said: ""We are certainly not on the de..."
1,Isinbayeva claims new world best\n\nPole vault...,"In the men's 60m, former Olympic 100m champion..."
2,Klinsmann issues Lehmann warning\n\nGermany co...,"Klinsmann added: ""If he is not playing regular..."
3,Hantuchova in Dubai last eight\n\nDaniela Hant...,Williams was also far from content.Davenport d...
4,Greek duo cleared in doping case\n\nSprinters ...,"Kenteris's lawyer, Gregory Ioannidis, said: ""T..."


In [9]:
data['Title']= "sports"
data.head()

,News,Summary,Title
0,Robinson answers critics\n\nEngland captain Ja...,"Robinson said: ""We are certainly not on the de...",sports
1,Isinbayeva claims new world best\n\nPole vault...,"In the men's 60m, former Olympic 100m champion...",sports
2,Klinsmann issues Lehmann warning\n\nGermany co...,"Klinsmann added: ""If he is not playing regular...",sports
3,Hantuchova in Dubai last eight\n\nDaniela Hant...,Williams was also far from content.Davenport d...,sports
4,Greek duo cleared in doping case\n\nSprinters ...,"Kenteris's lawyer, Gregory Ioannidis, said: ""T...",sports


In [10]:
data.isna().sum()

News       0
Summary    0
Title      0
dtype: int64

In [11]:
final_dataset= Dataset.from_pandas(data)
final_dataset

Dataset({
    features: ['News', 'Summary', 'Title'],
    num_rows: 511
})

In [12]:
nlp_data = final_dataset.train_test_split(test_size=0.2)
nlp_data

DatasetDict({
    train: Dataset({
        features: ['News', 'Summary', 'Title'],
        num_rows: 408
    })
    test: Dataset({
        features: ['News', 'Summary', 'Title'],
        num_rows: 103
    })
})

In [13]:
checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/home/p_r_prod/miniconda3/envs/text2image/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [14]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["News"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["Summary"], max_length=224, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
tokenized_dataset = nlp_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/103 [00:00<?, ? examples/s]

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [18]:
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [19]:
training_args = Seq2SeqTrainingArguments(
    output_dir="Fine-tuned-t5-text-summarizer",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [20]:
trainer = Seq2SeqTrainer(
    model=model,
    args= training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)

In [22]:
trainer.train() 

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/p_r_prod/miniconda3/envs/text2image/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.530813,0.183700,0.151400,0.176000,0.175600,19.000000
2,No log,0.471549,0.182300,0.153000,0.175300,0.175500,19.000000
3,No log,0.457654,0.181300,0.150000,0.173500,0.173500,19.000000
4,No log,0.452696,0.181700,0.154400,0.174500,0.174600,19.000000
5,No log,0.445636,0.186800,0.159500,0.180100,0.180500,19.000000
6,No log,0.446514,0.177000,0.148500,0.169000,0.169400,19.000000
7,No log,0.450587,0.190600,0.165200,0.184200,0.184400,19.000000
8,No log,0.456941,0.185400,0.160100,0.179200,0.179200,19.000000
9,No log,0.461430,0.188900,0.162200,0.182300,0.182500,19.000000
10,0.505600,0.466810,0.184000,0.156700,0.176700,0.176800,19.000000


/home/p_r_prod/miniconda3/envs/text2image/lib/python3.9/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/p_r_prod/miniconda3/envs/text2image/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/p_r_prod/miniconda3/envs/text2image/lib/python3.9/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/p_r_prod/miniconda3/envs/text2image/lib/python3.9/site-packages/to

TrainOutput(global_step=2550, training_loss=0.17994484265645344, metrics={'train_runtime': 2982.2924, 'train_samples_per_second': 6.84, 'train_steps_per_second': 0.855, 'total_flos': 1.237094148685824e+16, 'train_loss': 0.17994484265645344, 'epoch': 50.0})

In [23]:
test_text= " summarize: Why is this deal historic?\
It is just another club for Lionel Messi to go to and show his skills but it is indeed \
historic for the MLS and American Football as a whole. This will bring them the most eyeballs \
they have ever seen. The social media following of Inter Miami has risen by more than 300% and \
the ticket price for all the matches of Inter Miami have gone from $50 to $10,000 . The amount \
of worldwide attention that the MLS will get from this one signing will be humongous.\
Lionel Messi will bring fans to the Stadiums and Inter Miami will have to move \
from their 18,000 seater stadium to the Hard Rock Café Stadium which has a capacity of 70,000 because \
that is the standard of Messi. The sponsors that both Inter Miami and the league will get are \
unfathomable. It is like Michael Jordan playing in the Spanish Basketball League. All the celebrities \
will line up to watch him play and new TV and streaming deals will be made with all the countries around \
the world like India."

In [24]:
tokenizer = AutoTokenizer.from_pretrained("pranjal0109/Fine-tuned-t5-text-summarizer")
inputs = tokenizer(test_text, return_tensors="pt").input_ids

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
model = AutoModelForSeq2SeqLM.from_pretrained("pranjal0109/Fine-tuned-t5-text-summarizer")
outputs = model.generate(inputs, max_new_tokens=500, do_sample=False)

In [26]:
summary= tokenizer.decode(outputs[0], skip_special_tokens=True)
summary

'The sponsors that both Inter Miami and the league will get are unfathomable.Lionel Messi will bring fans to the Stadiums and Inter Miami will have to move from their 18,000 seater stadium to the Hard Rock Café Stadium which has a capacity of 70,000 because that is the standard of Messi.The social media following of Inter Miami has risen by more than 300% and the ticket price for all the matches of Inter Miami have gone from $50 to $10,000.It is just another club for Lionel Messi to go to and show his skills but it is indeed historic for the MLS and American Football as a whole.'